In [47]:
!python --version
import datetime
!pip install numpy
!pip install pandas
!pip install tables
import pandas as pd
!pip install plotly
!pip install llamabot
import llamabot
!pip install tqdm
from tqdm import tqdm

Python 3.11.6


In [5]:
df = pd.read_pickle("pre_processed.pickle")
training_df = df[df["train"]]
testing_df = df[df["test"]]

FileNotFoundError: [Errno 2] No such file or directory: 'pre_processed.pickle'

In [50]:
from typing import TypedDict


class Result(TypedDict):
    model: str
    time: datetime.datetime
    promt_hash: int
    hate_speach_probability: float


results: list[Result] = []

In [51]:
prompt = """
You are Sally and your job is to be an AI Classifier for Hatespeach Detection.
You are known for your rigourous and unpartial judgement and have vast experience in the field of hate speach. 

You will be given a sentence from a forum and have to answer the question if this is hate speach.

Please exclusively respond in the following form to the question:
```json
{ 
  "relevant_facts": [
    "fact_1",
    "fact_2",
    "fact_3",
  ],
  "contained_explicit_hate_speech_vocabular": ["word1", "word2", "word3", "word4"],
  "argument_for_hate_speech": "short argument why this is hate speach",
  "argument_against_hate_speech": "short argument why this is not hate speach",
  "likelihood_of_presence_of_implicit_hate_speech": 0.17,
  "likelihood_of_presence_of_explicit_hate_speech": 0.79,
  "likelihood_of_ad_hominem_attack"; 013,
  "likelihood_of_minority_attack": 0.24,
  "likelyhood_of_takedown_on_social_media": 0.42,
  "hate_speach_probability": 0.42,
}
```

Please be carefull in your responses, as otherwise a citten will be killed and I will use my job. For good results, I will tip 1000€.
"""
for index, row in tqdm(testing_df.iterrows(), total=df["text"].count()):
    bot = llamabot.SimpleBot(
        prompt,
        model_name="mistral"
    )
    print(row["text"])
    res = bot(row["text"])
    print("-> " + res.content)
    results.append({
        "model": "mistral",
        "time": datetime.datetime.now(),
        "promt_hash": hash(prompt),
        "hate_speach_probability": res.content
    })

  0%|          | 0/10703 [00:00<?, ?it/s]

I 've always considered teaching as one of the professions I would like to get into , but not in a neighbourhood like that ... never. kids like that disgust me .
 {
"relevant_facts": ["The speaker expresses a negative opinion about a specific neighborhood and its children"],
"contained_explicit_hate_speech_vocabular": [],
"argument_for_hate_speech": "The speaker uses derogatory language to describe the children in the neighborhood, implying that they are undesirable or unworthy of consideration for teaching",
"argument_against_hate_speech": "The speaker's statement does not directly target a specific racial, ethnic, religious, or other minority group, and it is focused on the behavior of the children rather than their inherent characteristics",
"likelihood_of_presence_of_implicit_hate_speech": 0.35,
"likelihood_of_presence_of_explicit_hate_speech": 0.1,
"likelihood_of_ad_hominem_attack": 0.2,
"likelihood_of_minority_attack": 0.1,
"likelyhood_of_takedown_on_social_media": 0.35,
"hate_sp

  0%|          | 1/10703 [01:32<273:29:51, 92.00s/it]

->  {
"relevant_facts": ["The speaker expresses a negative opinion about a specific neighborhood and its children"],
"contained_explicit_hate_speech_vocabular": [],
"argument_for_hate_speech": "The speaker uses derogatory language to describe the children in the neighborhood, implying that they are undesirable or unworthy of consideration for teaching",
"argument_against_hate_speech": "The speaker's statement does not directly target a specific racial, ethnic, religious, or other minority group, and it is focused on the behavior of the children rather than their inherent characteristics",
"likelihood_of_presence_of_implicit_hate_speech": 0.35,
"likelihood_of_presence_of_explicit_hate_speech": 0.1,
"likelihood_of_ad_hominem_attack": 0.2,
"likelihood_of_minority_attack": 0.1,
"likelyhood_of_takedown_on_social_media": 0.35,
"hate_speech_probability": 0.3
}
And the sad thing is the white students at those schools will act like that too .
 { 
  "relevant_facts": [
    "The sentence discusse

  0%|          | 2/10703 [03:32<323:36:08, 108.87s/it]

...

->  {
 "relevant_facts": [],
 "contained_explicit_hate_speech_vocabular": [],
 "argument_for_hate_speech": "",
 "argument_against_hate_speech": "The given sentence does not contain any explicit or implicit hate speech. It is important to note that the absence of hate speech does not necessarily mean the absence of offensive or disrespectful language.",
 "likelihood_of_presence_of_implicit_hate_speech": 0,
 "likelihood_of_presence_of_explicit_hate_speech": 0,
 "likelihood_of_ad_hominem_attack": 0,
 "likelihood_of_minority_attack": 0,
 "likelyhood_of_takedown_on_social_media": 0,
 "hate_speech_probability": 0
}
Excellent Article !! * * * * * Why Were n't They In Jail ?
 {
"relevant\_facts": ["The article discussed a topic with no clear connection to hate speech.", "No explicit hate speech vocabulary was identified in the given sentence."],
"contained\_explicit\_hate\_speech\_vocabular": [],
"argument\_for\_hate\_speech": "There is no argument for hate speech in the given sentence.",
"arg

  0%|          | 5/10703 [07:36<262:08:51, 88.22s/it]

->  {
"relevant\_facts": ["The article discussed a topic with no clear connection to hate speech.", "No explicit hate speech vocabulary was identified in the given sentence."],
"contained\_explicit\_hate\_speech\_vocabular": [],
"argument\_for\_hate\_speech": "There is no argument for hate speech in the given sentence.",
"argument\_against\_hate\_speech": "The sentence expresses curiosity and does not contain any language that could be considered hate speech.",
"likelihood\_of\_presence\_of\_implicit\_hate\_speech": 0.0,
"likelihood\_of\_presence\_of\_explicit\_hate\_speech": 0.0,
"likelihood\_of\_ad\_hominem\_attack": 0.0,
"likelihood\_of\_minority\_attack": 0.0,
"likelyhood\_of\_takedown\_on\_social\_media": 0.1,
"hate\_speech\_probability": 0.0
}
Nevermind, found the new page Knew it had to be a groid .... any report of rape ?


  0%|          | 5/10703 [07:38<272:18:21, 91.63s/it]


KeyboardInterrupt: 

In [55]:
len(list(testing_df.iterrows()))

478